In [2]:
import pandas as pd;
import random
import numpy as np;
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [2]:
path_test = 'data/credit_test.csv'
path_train = 'data/credit_train.csv'

In [3]:
#надеюсь в следующий раз огранизаторы предоставят данные в нормальной кодировке
test = pd.read_csv(path_test,sep =  ';',encoding = "cp1251")
train = pd.read_csv(path_train,sep =  ';',encoding = "cp1251")

In [4]:
train['test'] = 0
test['test'] = 1

In [5]:
#ссоединяем test и train, чтобы одновременно их обрабатывать
data = pd.DataFrame()
data = train.copy()
test['open_account_flg'] = -1
data = data.append(test, ignore_index=True)

In [6]:
data['sex'] = (data['gender'] == 'M').astype(np.int8)
del data['gender']

для всех номинальных признаков юзаем One-hot

In [7]:
data['m_UNM'] = (data['marital_status'] == 'UNM').astype(np.int8)
data['m_DIV'] = (data['marital_status'] == 'DIV').astype(np.int8)
data['m_MAR'] = (data['marital_status'] == 'MAR').astype(np.int8)
data['m_WID'] = (data['marital_status'] == 'WID').astype(np.int8)
data['m_CIV'] = (data['marital_status'] == 'CIV').astype(np.int8)
del data['marital_status']

In [8]:
#работа, причем в данных оказалось на несколько позиций больше, чем было в описании 
data['j_SPC'] = (data['job_position'] == 'SPC').astype(np.int8)
data['j_UMN'] = (data['job_position'] == 'UMN').astype(np.int8)
data['j_BIS'] = (data['job_position'] == 'BIS').astype(np.int8)
data['j_PNA'] = (data['job_position'] == 'PNA').astype(np.int8)
data['j_DIR'] = (data['job_position'] == 'DIR').astype(np.int8)
data['j_ATP'] = (data['job_position'] == 'ATP').astype(np.int8)
data['j_WRK'] = (data['job_position'] == 'WRK').astype(np.int8)
data['j_NOR'] = (data['job_position'] == 'NOR').astype(np.int8)
data['j_WOI'] = (data['job_position'] == 'WOI').astype(np.int8)
data['j_INP'] = (data['job_position'] == 'INP').astype(np.int8)
data['j_other'] = data['job_position'].isin(['BIU','WRP','PNI','PNV','HSK','PNS','INV','ONB'])
data['j_other'] = data['j_other'].astype(np.int8)
del data ['job_position']

In [9]:
#образование 
data['e_SCH'] = (data['education'] == 'SCH').astype(np.int8)
data['e_PGR'] = (data['education'] == 'PGR').astype(np.int8)
data['e_GRD'] = (data['education'] == 'GRD').astype(np.int8)
data['e_UGR'] = (data['education'] == 'UGR').astype(np.int8)
data['e_ACD'] = (data['education'] == 'ACD').astype(np.int8)
del data['education']

In [10]:
#бинаризуем тариф 
#до 1.26 тарифа норм, а остальное мелочь 
tariff = data['tariff_id'].value_counts().index[:-8]
for i in tariff:
    a = 't_' + str(i)
    data[a] = (data['tariff_id'] == i).astype(np.int8)
data['t_other'] = False
data.loc[(data[data['tariff_id'].isin(data['tariff_id'].value_counts().index[-8:])]['t_other'] == True).index,'t_other'] = True
data['t_other'] = data['t_other'].astype(np.int8)
del data['tariff_id']

In [11]:
#преобразуем credit_sum в int, также добавим колонку частотности признака
data['credit_sum'] = data['credit_sum'].apply(lambda x: float(x.replace(',', '.')))
credit_sum_freq = data['credit_sum'].value_counts(normalize=True).to_dict()
data['credit_sum_Freq'] = data['credit_sum'].apply(lambda x: credit_sum_freq[x])

In [12]:
#т.к кредит не наличкой, а на покупку вещи, то добавим признаки
# на что заканчивается цена покупки
data['is_990'] = (data['credit_sum'] % 1000 == 990).astype(np.int8)
data['is_int'] = (data['credit_sum'] % 1 == 0).astype(np.int8)
data['is_89'] = (data['credit_sum'] % 100 == 89).astype(np.int8)
data['is_68'] = (data['credit_sum'] % 100 == 68).astype(np.int8)
data['is_69'] = (data['credit_sum'] % 100 == 69).astype(np.int8)
data['is_79'] = (data['credit_sum'] % 100 == 79).astype(np.int8)
data['is_9'] = (data['credit_sum'] % 10 == 9).astype(np.int8)
data['is_8'] = (data['credit_sum'] % 10 == 8).astype(np.int8)
data['is_0'] = (data['credit_sum'] % 10 == 0).astype(np.int8)
data['is_7'] = (data['credit_sum'] % 10 == 7).astype(np.int8)
data['is_1'] = (data['credit_sum'] % 10 == 1).astype(np.int8)

In [13]:
#One-hot для кол-ва месяцев, также частотность признака
data['credit_month'] = data['credit_month'].astype(np.int8)
credit_month_freq = data['credit_month'].value_counts(normalize=True).to_dict()
data['credit_month_Freq'] = data['credit_month'].apply(lambda x: credit_month_freq[x])

In [14]:
#так как кодировка была плохая, то числа float были разделены запятой, а не точкой, поэтому pandas не определял их тип правильно
data['score_shk'] = data['score_shk'].apply(lambda x: float(x.replace(',', '.')))
data['score_shk_rare1'] = (data['score_shk'] <= data['score_shk'].quantile(0.01)).astype(np.int8)
data['score_shk_rare99'] = (data['score_shk'] >= data['score_shk'].quantile(0.99)).astype(np.int8)
data['score_shk'] = data['score_shk'].astype(np.float32)

In [15]:
data['age'] = data['age'].astype(np.int8)
age_freq = data['age'].value_counts(normalize=True).to_dict()
data['age_Freq'] = data['age'].apply(lambda x: age_freq[x])

In [16]:
#One-hot для дохода, также частотность признака
data.loc[data[pd.isnull(data['monthly_income'])].index,'monthly_income'] = data['monthly_income'].mean()
data['monthly_income'] = data['monthly_income'].astype(np.int32)
data['incme_round1000'] = (data['monthly_income'] % 1000 == 0).astype(np.int8)
data['incme_round100'] = (data['monthly_income'] % 100 == 0).astype(np.int8)
data['incme_round10'] = (data['monthly_income'] % 10 == 0).astype(np.int8)
monthly_income_freq = data['monthly_income'].value_counts(normalize=True).to_dict()
data['monthly_income_Freq'] = data['monthly_income'].apply(lambda x: monthly_income_freq[x])

In [17]:
#были пропуски в credit_count и overdue_credit_count
#для overdue_credit_count заполним модой, а для credit_count сделаем простую линейную регрессию
#я что-то тогда был нубом, и не знал что  StandardScaler надо юзать для линейной регрессии, поэтому без StandardScaler
# также добавлен is_nun -  признак был ли пропуск или не было 
data['is_nun'] = False
data.loc[data[data['credit_count'].isnull()].index,'is_nun'] = True
data['is_nun'] = data['is_nun'].astype(np.int8)
data.loc[data[data['overdue_credit_count'].isnull()].index,'overdue_credit_count'] = data['overdue_credit_count'].quantile()
data['overdue_credit_count'] = data['overdue_credit_count'].astype(np.int8)

In [18]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
X = data.ix[~data['credit_count'].isnull(), ~data.columns.isin(['credit_count','living_region'])]
y = data.ix[~data['credit_count'].isnull(), 'credit_count']
lr.fit(X, y)
v = lr.predict(data.ix[data['credit_count'].isnull(),~data.columns.isin(['credit_count','living_region'])])
data.ix[data['credit_count'].isnull(), 'credit_count'] = v.astype(np.int8)

/Users/romankucev/anaconda/lib/python2.7/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [20]:
data['credit_count'] = data['credit_count'].astype(np.int8)
credit_count_freq = data['credit_count'].value_counts(normalize=True).to_dict()
data['credit_count_Freq'] = data['credit_count'].apply(lambda x: credit_count_freq[x])

In [22]:
#вот тут я тупанул и скорее всего из-за этого не затащил конкурс(((
#в данной задаче от написания региона зависит таргет, поэтому подобной нормализацией только ухудшаете ситуацию.
#Точнее, этот столбец можно добавить дополнительно, вторым, но не вместо.
# т е я "почистил" регионы, убив много информации
region = data['living_region'].copy()
region = region.str.encode("utf8")
region[region.isnull()] = 'Missing'
region = region.str.replace('ОБЛ ', '')
region = region.str.replace('КРАЙ ', '')
region = region.str.replace('РЕСПУБЛИКА ', '')
region = region.str.replace('РЕСП ', '')
region = region.str.replace('ОБЛАСТЬ ', '')
region = region.str.replace('ОБЛ ', '')
region = region.str.replace(' КРАЙ', '')
region = region.str.replace(' РЕСПУБЛИКА', '')
region = region.str.replace(' РЕСП', '')
region = region.str.replace(' ОБЛАСТЬ', '')
region = region.str.replace(' ОБЛ', '')
region = region.str.replace(' ГОРОД', '')
region = region.str.replace(' Г', '')
region = region.str.replace('РЕСП. ', '')
region = region.str.replace(' Р-Н', '')
region = region.str.replace(' АО', '')
region = region.str.replace('Г. ', '')
region = region.str.replace('ОБЛ. ', '')
region = region.str.replace('ОБЛ.', '')
region = region.str.replace('КРАЙ. ', '')
region = region.str.replace(' АO', '')
region = region.str.replace('АО ', '')
region = region.str.replace('Г ', '')
region = region.str.replace('98', 'САНКТ-ПЕТЕРБУРГ')
region = region.str.replace('74', 'ЧЕЛЯБИНСКАЯ')
region = region.str.replace('ОДИНЦОВО ', '')

In [23]:
for i in region.index:
    region[i] = region[i].replace('Г.','')
    region[i] = region[i].replace('РЕСП.','')
    region[i] = region[i].replace('КРАЙ.','')
    region[i] = region[i].replace('КАМЧАТС??ИЙ','КАМЧАТСКИЙ')
    region[i] = region[i].replace('ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУ- ЮГРА','ХАНТЫ-МАНСИЙСКИЙ')
    region[i] = region[i].replace('ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУ- Ю','ХАНТЫ-МАНСИЙСКИЙ')
    region[i] = region[i].replace('ХАНТЫ-МАНСИЙСКИЙ-ЮГРА','ХАНТЫ-МАНСИЙСКИЙ')
    region[i] = region[i].replace('САХА /ЯКУТИЯ/','САХА')
    region[i] = region[i].replace('САХА (ЯКУТИЯ)','САХА')
    region[i] = region[i].replace('СЕВ.','СЕВЕРНАЯ')
    region[i] = region[i].replace('ЧУВАШСКАЯ - ЧУВАШИЯ','ЧУВАШИЯ ЧУВАШСКАЯ')
    region[i] = region[i].replace('СЕВ.','СЕВЕРНАЯ')
    region[i] = region[i].replace('ЕВРЕЙСКАЯБЛ','ЕВРЕЙСКАЯ АВТОНОМНАЯ')
    region[i] = region[i].replace('АЕВРЕЙСКАЯ','ЕВРЕЙСКАЯ АВТОНОМНАЯ')
    region[i] = region[i].replace('РОССИЯ','Missing')
    region[i] = region[i].replace('МОСКВОСКАЯ','МОСКОВСКАЯ')
    region[i] = region[i].replace('МЫТИЩИНСКИЙ','МОСКОВСКАЯ')
    region[i] = region[i].replace('РЕСПУБЛИКАТАТАРСТАН','ТАТАРСТАН')
    region[i] = region[i].replace('ЧУВАШИЯ ЧУВАШСКАЯ -','ЧУВАШИЯ ЧУВАШСКАЯ')
    region[i] = region[i].replace('КАМЧАТСКАЯ','КАМЧАТСКИЙ')
    region[i] = region[i].replace('ЧЕЛЯБИНСК','Missing')
    region[i] = region[i].replace('ОРЁЛ','ОРЛОВСКАЯ')
    region[i] = region[i].replace('ПРИВОЛЖСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ','Missing')
    region[i] = region[i].replace('БРЯНСКИЙ','БРЯНСКАЯ')
    region[i] = region[i].replace('МОСКОВСКИЙ П','МОСКОВСКАЯ')
    region[i] = region[i].replace('ЭВЕНКИЙСКИЙ','Missing')
    region[i] = region[i].replace('ГУСЬ-ХРУСТАЛЬНЫЙ','Missing')
    region[i] = region[i].replace('ДАЛЬНИЙ ВОСТОК','Missing')
    region[i] = region[i].replace('ПЕРМСКАЯ','ПЕРМСКИЙ')
    region[i] = region[i].replace('ЧУВАШИЯ ЧУВАШСКАЯ','ЧУВАШСКАЯ')
    
    if region[i][-1:] == '.':
        region[i] = region[i][0:-1]
data['living_region']  =  region  
data['r_iy'] = data['living_region'].str.endswith('ИЙ').astype(np.int8)
data['r_aya'] = data['living_region'].str.endswith('АЯ').astype(np.int8)

In [24]:
#ручками добавил в region.txt среднюю зарплату по регионам и добавил как фичу в данные
#dat = pd.DataFrame()
#dat['living_region'] = data['living_region'].value_counts().index
#dat['count'] = ' '
#dat.to_csv('region.txt',sep = ';')
dat = pd.read_csv('region.txt',sep = ';',index_col = 1)
dat['coun'] = dat['count'].apply(lambda x: float(x.replace(',', '.')))
dat = dat['coun']
data['salary_region'] = data['living_region'].apply(lambda x: dat[x])
data['salary_region'] = data['salary_region'] * 1000

In [25]:
#также добавил кол-во населения в регионе
dat = pd.DataFrame()
#dat['living_region'] = data['living_region'].value_counts().index
#dat['count'] = ' '
#dat.to_csv('region_popular.txt',sep = ';')
dat = pd.read_csv('region_popular.txt',sep = ';',index_col = 1)
dat['coun'] = dat['count'].apply(lambda x: float(x.replace(',', '.')))
dat = dat['coun']
data['region_popular'] = data['living_region'].apply(lambda x: dat[x])

In [26]:
#средний процент взятых кредитов тинькова для каждого региона
data['approval_rate'] = 0
dtr = data[data['open_account_flg'] != -1]
for txt in dtr['living_region'].value_counts().index:
    data.loc[data[data['living_region'] == txt].index,'approval_rate']  = dtr[dtr['living_region'] == txt]['open_account_flg'].sum() / float(len(dtr[dtr['living_region'] == txt]))

In [27]:
data['credit_to_income'] = data['credit_sum'] / data['monthly_income']
data['credit_to_income_m'] = data['credit_sum'] / (data['monthly_income'] * data['credit_month'])
data['credit_to_salary_region'] = data['credit_sum'] / data['salary_region']
data['income_to_salary_region'] = data['monthly_income'] / data['salary_region']

In [28]:
#one-hot для регионов, сначала хотел взять только самые частые регионы, а потом решли бинаризовать все регионы
reg = data[u'living_region'].value_counts().index[:-1]
for i in reg:
    a = u'reg_' + str(i).decode("utf8")
    data[a] = (data[u'living_region'] == i).astype(np.int8)
data[u'reg_other'] = False
data.loc[(data[data[u'living_region'].isin(data[u'living_region'].value_counts().index[-1:])][u'reg_other'] == True).index,u'reg_other'] = True
data[u'reg_other'] = data[u'reg_other'].astype(np.int8)
del data['living_region']

In [30]:
#оптимизируем память
for col in data.columns:
    if set(data[col]) == set((0,1)):
        data[col] = data[col].astype(np.int8)

In [31]:
del data['open_account_flg']

In [32]:
new_test = data[data['test'] == 1].copy()
new_train = data[data['test'] == 0].copy()
del new_test['test']
del new_train['test']

In [33]:
new_train.to_pickle('data/train_prep.pkl')
new_test.to_pickle('data/test_prep.pkl')